In [1]:
chars = []
counts = []

with open("./data/day12.txt") as f:
    while line := f.readline():
        line = line.rstrip().split()
        chars.append(line[0])
        counts.append(list(map(int, line[1].split(','))))

In [2]:
from functools import cache

@cache
def dp(list_ind, char_ind, count_ind):
    if count_ind >= len(counts[list_ind]): # no more #'s to fill
        if '#' in chars[list_ind][char_ind:]:
            return 0 # and no more #'s in the remaining string
        return 1 # but #'s remaining in the string
    if char_ind >= len(chars[list_ind]):
        return 0 # no characters left but we still need to fill #'s
    if chars[list_ind][char_ind] == '.':
        return dp(list_ind, char_ind+1, count_ind)
    if chars[list_ind][char_ind] == '#':
        if counts[list_ind][count_ind] > len(chars[list_ind])-char_ind:
            return 0 # we need to fill #'s but we dont have enough characters left
        if '.' in chars[list_ind][char_ind:char_ind+counts[list_ind][count_ind]]:
            return 0 # there is a . in the way
        if char_ind+counts[list_ind][count_ind] < len(chars[list_ind]) and chars[list_ind][char_ind+counts[list_ind][count_ind]] == '#':
            return 0 # the fill of #'s would immediately continue with a #
        return dp(list_ind, char_ind+counts[list_ind][count_ind]+1, count_ind+1)
    res = dp(list_ind, char_ind+1, count_ind)
    if counts[list_ind][count_ind] > len(chars[list_ind])-char_ind:
        return res
    if '.' in chars[list_ind][char_ind:char_ind+counts[list_ind][count_ind]]:
        return res
    if char_ind+counts[list_ind][count_ind] < len(chars[list_ind]) and chars[list_ind][char_ind+counts[list_ind][count_ind]] == '#':
        return res 
    return res+dp(list_ind, char_ind+counts[list_ind][count_ind]+1, count_ind+1)

In [3]:
s = 0
for i in range(len(chars)):
    res = dp(i, 0, 0)
    s += res
s

6871

In [4]:
dp.cache_info()

CacheInfo(hits=4305, misses=20991, maxsize=None, currsize=20991)

In [5]:
dp.cache_clear()

In [6]:
chars = []
counts = []

with open("./data/day12.txt") as f:
    while line := f.readline():
        line = line.rstrip().split()
        chars.append('?'.join([line[0], line[0], line[0], line[0], line[0]]))
        counts.append(list(map(int, line[1].split(',')))*5)

In [7]:
print(chars[0])

#???#?#?.?##?#?.#.?#???#?#?.?##?#?.#.?#???#?#?.?##?#?.#.?#???#?#?.?##?#?.#.?#???#?#?.?##?#?.#.


In [8]:
print(counts[0])

[7, 5, 1, 7, 5, 1, 7, 5, 1, 7, 5, 1, 7, 5, 1]


In [9]:
s = 0
for i in range(len(chars)):
    res = dp(i, 0, 0)
    s += res
s

2043098029844

# Recursion with memoization vs 2D DP

In [10]:
def actual_dp():
    s = 0
    for list_ind in range(len(chars)):
        dp = [0]*(len(chars[list_ind])+2)
        dp[-1] = dp[-2] = 1
        for char_ind in range(len(chars[list_ind])-1, -1, -1):
            if chars[list_ind][char_ind] == '#':
                break
            dp[char_ind] = 1
        for count_ind in range(len(counts[list_ind])-1, -1, -1):
            new_dp = [0]*(len(chars[list_ind])+2)
            for char_ind in range(len(chars[list_ind])-1, -1, -1):
                if chars[list_ind][char_ind] == '.':
                    new_dp[char_ind] = new_dp[char_ind+1]
                    continue
                if chars[list_ind][char_ind] == '#':
                    if counts[list_ind][count_ind] > len(chars[list_ind])-char_ind:
                        new_dp[char_ind] = 0
                    elif '.' in chars[list_ind][char_ind:char_ind+counts[list_ind][count_ind]]:
                        new_dp[char_ind] = 0
                    elif char_ind+counts[list_ind][count_ind] < len(chars[list_ind]) and chars[list_ind][char_ind+counts[list_ind][count_ind]] == '#':
                        new_dp[char_ind] = 0
                    else:
                        new_dp[char_ind] = dp[char_ind+counts[list_ind][count_ind]+1]
                    continue
                res = new_dp[char_ind+1]
                if counts[list_ind][count_ind] > len(chars[list_ind])-char_ind:
                    new_dp[char_ind] = res
                elif '.' in chars[list_ind][char_ind:char_ind+counts[list_ind][count_ind]]:
                    new_dp[char_ind] = res
                elif char_ind+counts[list_ind][count_ind] < len(chars[list_ind]) and chars[list_ind][char_ind+counts[list_ind][count_ind]] == '#':
                    new_dp[char_ind] = res
                else:
                    new_dp[char_ind] = res+dp[char_ind+counts[list_ind][count_ind]+1]
            dp = new_dp
        s += dp[0]
    return s

In [11]:
actual_dp()

2043098029844

In [12]:
%%timeit -n 5

actual_dp()

501 ms ± 3.89 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [13]:
%%timeit -n 5

dp.cache_clear()

s = 0
for i in range(len(chars)):
    res = dp(i, 0, 0)
    s += res
s

189 ms ± 1.86 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)
